## HW6

## Problem 2 (1)

#### In this problem, I used the library from libsvm.

#### For the following libsvm parameters,<br>
#### -t: 0 for linear kernel, 1 for polynomial<br>
#### -v:10 for 10 fold cross validation
#### -d:5 for 5 degree

In [2]:
import libsvm.python.svmutil as SVM

#### load train_x, train_y, test_x, test_y

In [3]:
y1, x1 = SVM.svm_read_problem('DogsVsCats/DogsVsCats.train')

In [4]:
y2, x2 = SVM.svm_read_problem('DogsVsCats/DogsVsCats.test')

# Linear

### Validation

In [5]:
m1 = SVM.svm_train(y1,x1,'-t 0 -v 10')

Cross Validation Accuracy = 59.904%


### Training

In [153]:
m2 = SVM.svm_train(y1,x1,'-t 0')

In [154]:
p_label, p_acc, p_val = SVM.svm_predict(y1, x1, m2)

Accuracy = 60.12% (7515/12500) (classification)


### Testing

In [155]:
p_label, p_acc, p_val = SVM.svm_predict(y2, x2, m2)

Accuracy = 59.2% (7400/12500) (classification)


## Polynomial

In [13]:
y1, x1 = SVM.svm_read_problem('DogsVsCats/DogsVsCats.train')

In [14]:
y2, x2 = SVM.svm_read_problem('DogsVsCats/DogsVsCats.test')

### Validation

In [15]:
m3 = SVM.svm_train(y1,x1,'-t 1 -v 10 -d 5')

Cross Validation Accuracy = 50.032%


### Training

In [16]:
m4 = SVM.svm_train(y1,x1,'-t 1 -d 5')

In [17]:
p_label, p_acc, p_val = SVM.svm_predict(y1, x1, m4)

Accuracy = 50.024% (6253/12500) (classification)


### Testing

In [18]:
p_label, p_acc, p_val = SVM.svm_predict(y2, x2, m4)

Accuracy = 50.048% (6256/12500) (classification)


## Problem 2 (2)

In [238]:
import pandas as pd
import math
import random

#### Combine the feature dictionary with the label list to form a pandas DataFrame

In [122]:
def combine_columns(x,y):
    temp1 = pd.DataFrame(x)
    temp1['output']=y
    return temp1

#### Add initial weight for each new round of boosting
#### weight = 1 / number of rows

In [123]:
def add_initial_weight(df):
    df['weight']= 1/df.shape[0]

#### Calculate the Total Error
#### Total Error = SUM of all wrong prediction's weight

In [124]:
def calculate_TE(df,l):
    count=0
    df['output2']=l
    for index, row in df.iterrows():
        if (row['output'] != row['output2']):
            count += row['weight']
    return count

#### Calculate the performance of stump
#### Performance of stump = 1/2 * ln[(1-total error)/total error)]

In [125]:
def calculate_perf_stump(te):
    return 1/2*math.log((1-te)/te)

#### Update the weight
#### New weight = old weight * e to the power of performance of stump
#### Normalize the new weight

In [248]:
def update_weight(df,p_stump):
    for index, rows in df.iterrows():
        if (rows['output'] != rows['output2']):
            rows['weight']=rows['weight'] * math.exp(p_stump)
        else:
            rows['weight']=rows['weight'] * math.exp(-p_stump)
    s = df.loc[:,'weight'].sum()
    df['weight'] = df['weight']/s
    return df

#### Add range with start and end to the dataframe
#### This will be used in the weight addition during bossting

In [212]:
def add_range(df):
    df['start']=''
    df['end']=''
    df.at[0,'start'] = 0
    df.at[0,'end']=df.at[0,'weight']
    for i in range (1,len(df)):
        df.at[i,'start'] = df.at[i-1,'end']
        df.at[i,'end']=df.at[i,'weight']+df.at[i,'start']
    return df

#### One round of boosting
#### Input with x_train,y_train,x_test,y_test
#### Ouput with new_x_train, new_y_train for the next round
#### We first train the model and get the p_label
#### Then we combine the x_test with y_test to form the test data set
#### We then perform the weight calculation and update the weight
#### We randomly select number in range (0-1) and select the row based on ramdom number >= start &<=end
#### We output the new_x_train and new_y_train

In [295]:
def get_new_train_test2(x_train,y_train,x_test,y_test):
    model = SVM.svm_train(y_train,x_train,'-t 0')
    p_label, p_acc, p_val = SVM.svm_predict(y_test, x_test, model)
    df = combine_columns(x_test,y_test)
    add_initial_weight(df)
    te = calculate_TE(df,p_label)
    p_s = calculate_perf_stump(te)
    df = add_range(update_weight(df,p_s))
    new_df = pd.DataFrame(columns=list(df.columns))
    for i in range(len(df)):
        x = random.uniform(0,1)
        new_df=new_df.append(df.loc[(df['end'] >= x) & (df['start'] <= x)])
    new_x_train = new_df.drop(['output','output2','weight','start','end'],axis=1).to_dict('records')
    new_y_train = new_df['output'].to_list()
    return new_x_train,new_y_train

#### Run the get_new_train_test2 method for k times

In [298]:
def adaboost2(x_train,y_train,x_test,y_test,k):
    for i in range(k):
        x_train,y_train = get_new_train_test2(x_train,y_train,x_test,y_test)

#### K = 10

In [299]:
adaboost2(x1,y1,x2,y2,10)

Accuracy = 59.2% (7400/12500) (classification)
Accuracy = 54.584% (6823/12500) (classification)
Accuracy = 58.128% (7266/12500) (classification)
Accuracy = 56.08% (7010/12500) (classification)
Accuracy = 55.552% (6944/12500) (classification)
Accuracy = 57.016% (7127/12500) (classification)
Accuracy = 54.728% (6841/12500) (classification)
Accuracy = 58.408% (7301/12500) (classification)
Accuracy = 54.776% (6847/12500) (classification)
Accuracy = 58.976% (7372/12500) (classification)


#### K = 20

In [300]:
adaboost2(x1,y1,x2,y2,20)

Accuracy = 59.2% (7400/12500) (classification)
Accuracy = 55.6% (6950/12500) (classification)
Accuracy = 57.168% (7146/12500) (classification)
Accuracy = 55.808% (6976/12500) (classification)
Accuracy = 55.272% (6909/12500) (classification)
Accuracy = 56.536% (7067/12500) (classification)
Accuracy = 55.776% (6972/12500) (classification)
Accuracy = 56.768% (7096/12500) (classification)
Accuracy = 54.736% (6842/12500) (classification)
Accuracy = 56.392% (7049/12500) (classification)
Accuracy = 54.896% (6862/12500) (classification)
Accuracy = 57.152% (7144/12500) (classification)
Accuracy = 54.984% (6873/12500) (classification)
Accuracy = 57.584% (7198/12500) (classification)
Accuracy = 55.128% (6891/12500) (classification)
Accuracy = 58.168% (7271/12500) (classification)
Accuracy = 54.856% (6857/12500) (classification)
Accuracy = 58.424% (7303/12500) (classification)
Accuracy = 54.64% (6830/12500) (classification)
Accuracy = 58.208% (7276/12500) (classification)
